In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from shutil import copy2 as copy
from IPython.display import display, HTML
from math import floor, ceil
import json
import re
import os

pd.set_option('display.max_rows', None)

secrets = json.load(open('secrets.json'))
game_version = secrets['game_version']
os.makedirs(game_version, exist_ok=True)

# Sacred Stones Weapons

In [ ]:
wp = pd.read_csv("data/weapons_ss.csv",sep='\s*?,\s*?', engine='python')
wp['Tags'] = wp.Tags.fillna('')

numeric = ['Mt','Wt']
# rng_map = {'1':1, '1~2':3, '2':2, '2~3':5, '1~3':4, '3~10':13, 'All':20, '1~Mag/A':10, '1~Mag/B':10,
#     '1~/Mag/B':10, '1~Mag/C':10, '1~Mag/D':10}
# rng_unmap = {1:'1', 3:'1~2', 2:'2', 5:'2~3', 4:'1~3', 13:'3~10', 20:'All', 10:'1~Str', 10:'1~Str',
#     10:'1~Str', 10:'1~Str', 10:'1~Str'}
# rng_norms = {"Swords": 1, "Lances": 1, "Axes": 1, "Bows": 2, "Reason": 3, "Dark": 3, "Faith": 3}

normal_wp = wp.loc[~wp.Tags.str.contains('Legendary') & (wp.Type != 'Monster')]

wp_tiers = normal_wp[['Name', 'Type']].copy()
for type in set(normal_wp['Type']):
    wp_tiers.loc[wp_tiers.Type == type, 'Mt'] = pd.cut(normal_wp.loc[wp.Type == type, 'Mt'], 5, retbins=True, labels=['0', '1', '2', '3', '4'])[0]
    wp_tiers.loc[wp_tiers.Type == type, 'Wt'] = pd.cut(normal_wp.loc[wp.Type == type, 'Wt'], 5, retbins=True, labels=['-2', '-1', '0', '1', '2'])[0]
wp_tiers['Mt'] = wp_tiers['Mt'].astype('str').str.replace('nan', '0')
wp_tiers['Wt'] = wp_tiers['Wt'].astype('str').str.replace('nan', '0')
wp_tiers = wp_tiers.merge(wp[['Name', 'Rng', 'DmgType', 'Tags']], on='Name').fillna('')

wp_tiers.loc[wp_tiers.Name == 'Iron Sword', 'Mt'] = '1' 
wp_tiers.loc[wp_tiers.Name == 'Shine', 'Wt'] = '-1' 

wp_tiers.sort_values('Mt')
wp_tiers[wp_tiers.Type=='Faith']

In [ ]:
export = wp_tiers.copy()
# export["Atk"] = export.apply(lambda x: x.Mag if x.Mag != '-' else x.Str, axis=1)
# export['HP'] = 20
export_order = ['Name', 'Type', 'Mt', 'Wt', 'Rng', 'DmgType', 'Tags']
export = export[export_order].sort_values('Type')
export.to_csv(f'{game_version}/weapons_ss.csv', index=False)
md_table = export.to_markdown(index=0)
open(f'{game_version}/Weapons.md', 'w').write(md_table)
open(f'{secrets["ObsidianRoot"]}/{game_version}/Weapons.md', 'w').write(md_table)
print(md_table)

# Engage Weapons

In [75]:
def fix(df, check, pattern, target, value):
    df.loc[df[check].str.contains(pattern), target] = value

def adjust(df, check, pattern, target, value):
    df.loc[df[check].str.contains(pattern), target] = df.loc[df[check].str.contains(pattern), target]+value


engage_raw = pd.read_csv("data/engage_weapons.csv",sep='\s*?,\s*?', engine='python')
engage_raw['Tags'] = engage_raw.Tags.fillna('')
numeric = ['Mt', 'Hit', 'Crit', 'Wt']
slicekey = 'Type'

# Raw data adjustments and filters

s_ranks = engage_raw.loc[(engage_raw.Lvl == 'S')]
engage_raw = engage_raw.loc[(engage_raw.Lvl != 'S') & (~engage_raw.Tags.str.contains('Exclusive'))]

engage_raw.loc[engage_raw.Name.str.contains("urge"), "Hit"] = 100
engage_raw.loc[engage_raw.Name.str.contains("Meteor"), "Wt"] = 15
engage_raw.loc[engage_raw.Mt > 24, "Mt"] = 26

# Rating generation

wp_tiers = engage_raw[['Name', slicekey]].copy()
for type in set(engage_raw[slicekey]):
    wp_tiers.loc[wp_tiers[slicekey] == type, 'Mt'] = pd.cut(engage_raw.loc[engage_raw[slicekey] == type, 'Mt'], 5, retbins=True, labels=['1', '3', '5', '7', '9'])[0]
    wp_tiers.loc[wp_tiers[slicekey] == type, 'Wt'] = pd.cut(engage_raw.loc[engage_raw[slicekey] == type, 'Wt'], 3, retbins=True, labels=['0', '-1', '-2'])[0]
wp_tiers['Mt'] = wp_tiers['Mt'].astype('str').str.replace('nan', '0')
wp_tiers['Wt'] = wp_tiers['Wt'].astype('str').str.replace('nan', '0')
wp_tiers['Hit'] = wp_tiers.loc[:, 'Hit'] = pd.cut(engage_raw.loc[:, 'Hit'], 5, retbins=True, labels=['2', '1', '0', '-1', '-2'])[0]
wp_data = wp_tiers.merge(engage_raw, on=[slicekey, 'Name'], suffixes=[None, 'Value']).fillna('')

s_ranks.Mt = s_ranks.apply(lambda x: 9 if int(x.Mt) > 19 else 7,axis=1)
s_ranks.loc[s_ranks.Name.str.contains('Nova|Art'), 'Mt'] = '1'
s_ranks.Wt = s_ranks.apply(lambda x: -2 if int(x.Wt) > 17 else -1,axis=1)
s_ranks.loc[s_ranks.Name.str.contains('Cinquedea'), 'Wt'] = '0'
s_ranks['HitValue'] = s_ranks['Hit']
s_ranks = s_ranks.fillna('')
wp_data = pd.concat([wp_data,s_ranks])

# Spot data tuning

wp_data.Mt = wp_data.Mt.astype('int')
wp_data.Wt = wp_data.Wt.astype('int')
wp_data.Tags = wp_data.apply(lambda x: x.Tags if int(x.HitValue) > 74 else x.Tags+f' Inaccurate({x.Hit})',axis=1)
wp_data.loc[wp_data.Name.str.contains("urge"), "Tags"] = wp_data.loc[wp_data.Name.str.contains("urge"), "Tags"] + 'Homing'
wp_data.loc[wp_data.Name.str.contains("Elsurge"), "Lvl"] = 'A'
wp_data.Price = wp_data.Price/100
wp_data.Tags = wp_data.apply(lambda x: x.Tags if not x.Uses else x.Tags+f' Consumable({ceil(x.Uses/2)})',axis=1)
wp_data["DmgType"] = wp_data.apply(lambda x: 'Mag' if x.Type in ['Tome'] else 'Phys',axis=1)
wp_data["DmgType"] = wp_data.apply(lambda x: 'Mag' if 'Enchanted' in x.Tags else x.DmgType,axis=1)

wp_data.loc[wp_data.Name.str.contains('Slim|Compact|Initiate|Short'), 'Mt'] = 0
adjust(wp_data, 'Name', '(Steel Great)', 'Mt', -2)
adjust(wp_data, 'Name', '(Killer Lance|Rider|Hammer|Poleaxe)', 'Mt', 2)
adjust(wp_data, 'Name', '(urge)', 'Mt', -2)
fix(wp_data, 'Name', 'Radiant', 'Mt', 7)
fix(wp_data, 'Name', 'Thunder', 'Mt', 3)
fix(wp_data, 'Name', 'Silver-Spirit', 'Mt', 5)
adjust(wp_data, 'Type', 'Dagger', 'Mt', -2)
adjust(wp_data, 'Type', 'Axe', 'Mt', 1)
wp_data.Mt = wp_data.Mt.clip(lower=0)
wp_data.loc[wp_data.Type.str.contains('Tome'), 'Mt'] = wp_data.loc[wp_data.Type.str.contains('Tome'), 'Mt'].clip(upper=7)
wp_data.loc[wp_data.Name.str.contains('Silver'), 'Lvl'] = 'A'
wp_data.loc[wp_data.Type.str.contains('Staff'), 'Mt'] = 0
wp_data.loc[wp_data.Type.str.contains('Staff'), 'Wt'] = 0

# Export 

wp_data.Wt = wp_data.Wt.astype('int')
export = wp_data.copy()
export_order = ['Type', 'Name', 'Lvl', 'Mt', 'Wt', 'Rng', 'DmgType', 'Price', 'Tags']
export = export[export_order].sort_values(['Type', 'Lvl'])

# ---

wp_data.sort_values(['Type', 'Mt'], ascending=True)
# wp_data[wp_data.Type == 'Art'].sort_values(['Mt'], ascending=True)
# wp_data[wp_data.Name.str.contains('Silver|Steel|Iron|Slim|Compact|Initiate|Short')].sort_values(['Type', 'Mt'], ascending=True)
# s_ranks

/var/folders/ks/dksml1ms0rg9dk9rjyqpnfdc0000gn/T/ipykernel_87134/4212574637.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df[check].str.contains(pattern), target] = df.loc[df[check].str.contains(pattern), target]+value


,Name,Type,Mt,Wt,Hit,MtValue,HitValue,Crit,WtValue,Rng,Lvl,Uses,Price,Tags,Notes/Description,DmgType
68,Initiate Art,Art,0,0,-1,5.0,95,0,5.0,1,D,,5.0,Nimble(10),Ddg+30; A scroll of a fundamental martial skil...,Phys
69,Iron-Body Art,Art,1,0,-1,2.0,90,0,3.0,1,D,,10.0,Brave,A scroll of a basic martial skill. If user ini...,Phys
87,Divine Fist Art,Art,1,-1,75,NaN,75,10,NaN,1,S,,100.0,Brave,A scroll of a legendary martial skill. If user...,Phys
72,Shielding Art,Art,3,-2,1,4.0,70,0,8.0,1,C,,25.0,Bonus(Def+1) Inaccurate(1),A scroll of a defensive martial skill. Grants ...,Phys
70,Steel-Hand Art,Art,5,0,1,5.0,70,5,5.0,1,C,,20.0,Brave Inaccurate(1),A scroll of a standard martial skill. If user ...,Phys
71,Silver-Spirit Art,Art,5,-1,0,8.0,80,0,7.0,1,A,,40.0,Brave,A scroll of a highly polished martial skill. I...,Phys
73,Flashing Fist Art,Art,9,-2,0,9.0,80,0,9.0,1,A,,60.0,Brave Bonus(Spd+1) Cursed(Def-2) Cursed(Res-2),A scroll of brash martial skill. Grants Spd+5;...,Phys
26,Compact Axe,Axe,1,0,-1,6.0,90,0,7.0,1,D,,5.0,Nimble(10),Ddg+20; A light; unusually accurate axe.,Phys
27,Iron Axe,Axe,2,0,0,9.0,80,0,10.0,1,D,,10.0,,A standard axe made of iron─simple; but effect...,Phys
30,Hand Axe,Axe,2,-1,1,7.0,70,0,12.0,1~2,D,,18.0,Inaccurate(1),A light throwing axe that can strike close or ...,Phys


In [72]:
export.to_csv(f'{game_version}/weapons_engage.csv', index=False)
md_table = export.to_markdown(index=0)
open(f'{game_version}/Weapons.md', 'w').write(md_table)
open(f'{secrets["ObsidianRoot"]}/{game_version}/Weapons.md', 'w').write(md_table)
# print(md_table)

13595

3.4